In [2]:
# import libray
import random
import spacy

from spacy.training import Example

from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL

In [3]:
# import nlp model
nlp = spacy.load('en_core_web_md')

In [28]:
# dataset for training model
food_comment = [("The pasta was perfectly cooked and full of flavor.",
                 {"cats" : {"Positive Sentiment" : 1.0, "Negative Sentiment" : 0.0}}),
                 ("I loved the crispy texture of the fried chicken.",
                 {"cats" : {"Positive Sentiment" : 1.0, "Negative Sentiment" : 0.0}}),
                 ("The dessert was rich and satisfying, I will definitely come back.",
                 {"cats" : {"Positive Sentiment" : 1.0, "Negative Sentiment" : 0.0}}),
                 ("The sushi was fresh and beautifully presented.",
                 {"cats" : {"Positive Sentiment" : 1.0, "Negative Sentiment" : 0.0}}),
                 ("Excellent service and the pizza tasted amazing!",
                 {"cats" : {"Positive Sentiment" : 1.0, "Negative Sentiment" : 0.0}}),
                 ("The burger was cold and lacked seasoning.",
                 {"cats" : {"Positive Sentiment" : 0.0, "Negative Sentiment" : 1.0}}),
                 ("I found the soup too salty and unpleasant to eat.",
                 {"cats" : {"Positive Sentiment" : 0.0, "Negative Sentiment" : 1.0}}),
                 ("The steak was overcooked and dry.",
                 {"cats" : {"Positive Sentiment" : 0.0, "Negative Sentiment" : 1.0}}),
                 ("The portion size was too small for the price.",
                 {"cats" : {"Positive Sentiment" : 0.0, "Negative Sentiment" : 1.0}}),
                 ("The bread was stale and the salad tasted bland.",
                 {"cats" : {"Positive Sentiment" : 0.0, "Negative Sentiment" : 1.0}})
                ]

In [5]:
# set config for model
config = {
    'threshold' : 0.5,
    'model' : DEFAULT_SINGLE_TEXTCAT_MODEL
}

In [6]:
# create the model object
text_cat = nlp.add_pipe('textcat', config=config)

In [7]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'textcat']

In [8]:
text_cat

In [29]:
# add two lablers to model
text_cat.add_label("Positive Sentiment")
text_cat.add_label("Negative Sentiment")

0

In [30]:
# create example
food_comment_exp = [Example.from_dict(nlp.make_doc(comment), category) for comment, category in food_comment]
text_cat.initialize(lambda: food_comment_exp, nlp=nlp)

In [31]:
# review
food_comment_exp

[{'doc_annotation': {'cats': {'Positive Sentiment': 1.0, 'Negative Sentiment': 0.0}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'spans': {}, 'links': {}}, 'token_annotation': {'ORTH': ['The', 'pasta', 'was', 'perfectly', 'cooked', 'and', 'full', 'of', 'flavor', '.'], 'SPACY': [True, True, True, True, True, True, True, True, False, False], 'TAG': ['', '', '', '', '', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', '', '', '', '', ''], 'POS': ['', '', '', '', '', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'DEP': ['', '', '', '', '', '', '', '', '', ''], 'SENT_START': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}},
 {'doc_annotation': {'cats': {'Positive Sentiment': 1.0, 'Negative Sentiment': 0.0}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'spans': {}, 'links': {}}, 'token_annotation': {'ORTH': ['I', 'loved', 'the', 'crispy', 'texture', 'of', 'the', 'fried', 'chicken', '.'], 'SPACY': [True

In [32]:
# training the model

# set epoch and losses
epochs = 20
losses = {}

# main loop for training
with nlp.select_pipes(enable='textcat'):
    optimizer = nlp.resume_training()
    for i in range(epochs):
        random.shuffle(food_comment)
        for comment, category in food_comment:
            mdoc = nlp.make_doc(comment)
            exp = Example.from_dict(mdoc, category)
            nlp.update([exp], sgd=optimizer, losses=losses)
        print('Epoch ',i, 'Losses ',losses)

Epoch  0 Losses  {'textcat': 2.5618902295827866}
Epoch  1 Losses  {'textcat': 4.274910882115364}
Epoch  2 Losses  {'textcat': 4.553515742067248}
Epoch  3 Losses  {'textcat': 4.555172721666111}
Epoch  4 Losses  {'textcat': 4.5551935526741225}
Epoch  5 Losses  {'textcat': 4.555198031053816}
Epoch  6 Losses  {'textcat': 4.555201213264223}
Epoch  7 Losses  {'textcat': 4.555202681378108}
Epoch  8 Losses  {'textcat': 4.555203833998733}
Epoch  9 Losses  {'textcat': 4.555204758281596}
Epoch  10 Losses  {'textcat': 4.555205510335684}
Epoch  11 Losses  {'textcat': 4.555206153927758}
Epoch  12 Losses  {'textcat': 4.55520669205619}
Epoch  13 Losses  {'textcat': 4.555207188054737}
Epoch  14 Losses  {'textcat': 4.555207623840325}
Epoch  15 Losses  {'textcat': 4.555208007058473}
Epoch  16 Losses  {'textcat': 4.555208353578716}
Epoch  17 Losses  {'textcat': 4.55520866919891}
Epoch  18 Losses  {'textcat': 4.5552089558136295}
Epoch  19 Losses  {'textcat': 4.555209213668125}


In [33]:
# test 1
test = nlp('The pizza was delicious')
test.cats

{'Positive Sentiment': 0.5669248700141907,
 'Negative Sentiment': 0.43307510018348694}

In [34]:
# test 2
test2 = nlp('The food not delicious and the service was terrible.')
test2.cats

{'Positive Sentiment': 0.005940255708992481,
 'Negative Sentiment': 0.9940598011016846}

In [35]:
# test3
test3 = nlp('The pancakes were fluffy and delicious.')
test3.cats

{'Positive Sentiment': 0.41497448086738586,
 'Negative Sentiment': 0.5850255489349365}

In [36]:
# test4
test4 = nlp('The food was cold and the service was slow.')
test4.cats

{'Positive Sentiment': 0.01045308355242014,
 'Negative Sentiment': 0.9895469546318054}

In [37]:
# test5
test5 = nlp('I enjoyed the fresh ingredients in the salad.')
test5.cats

{'Positive Sentiment': 0.6722080707550049,
 'Negative Sentiment': 0.3277919590473175}